# Resolving the Singularity in 2D

## 1. Setup / Rewrite functions for mpmath

In [ ]:
from mpmath import mp, mpf, acos, cos, pi, quad, sqrt, findroot, fabs
import matplotlib.pyplot as plt

t = 1
d = 2

mp.dps = 30

def heaviside(x):
    x = mpf(x)

    if x < 0:
        return 0
    else:
        return 1
    
def I_1(x):
    x = mpf(x)
    if fabs(x) <= 2 * t:
        result = heaviside(x + 2 * t) - (1 / pi) * acos(x / (2 * t))
    else:
        result = heaviside(x + 2 * t)
    return result

def I_2(x):
    x = mpf(x)
    
    def I_1_shifted(k):
        return I_1(x + 2 * t * cos(k))
    
    # Use mpmath's quad for integration
    integral_value = quad(I_1_shifted, [-pi, pi])
    
    return integral_value / (2 * pi)

def rho_hk_2d(mu, U):
    mu = mpf(mu)
    U = mpf(U)
    if U >= 0:
        return I_2(mu) + I_2(mu - U)
    else:
        return 2 * I_2(mu - U / 2)
    


def find_mu_of_rho_landau(rho, U, f_0):
    rho = mpf(rho)
    U = mpf(U)
    f_0 = mpf(f_0)

    lower = -2 * t * d
    upper = U + 2 * t * d + 2 * f_0
    bracket = (lower, upper)

    def func_mu(mu):
        return rho_hk_2d(mu - f_0 * rho, U) - rho

    try:
        mu_root = findroot(func_mu, bracket, solver='bisect', verbose=False, tol=mpf('1e-30'))
        return mu_root
    except Exception:
        raise RuntimeError(f"Keine Nullstelle gefunden für rho={rho}")
    
def create_mu_list(rho_array, U, f_0):
    mu_values = []
    for rho_i in rho_array:
        print(rho_i)
        try:
            mu = find_mu_of_rho_landau(rho_i, U, f_0)
            mu_values.append(mu)
        except RuntimeError:
            mu_values.append(mp.nan)
            print(f"Keine Nullstelle gefunden für rho={rho_i}")
    return mu_values



    
def DOS_HK_2D(E):
    E = mpf(E)

    def integrand_DOS_2d(k):
        arg = E / (2 * t) + cos(k)
        if -1 < arg < 1:
            return 1 / sqrt(1 - arg**2)
        else:
            return mpf(0)

    int_value = quad(integrand_DOS_2d, [0, pi])
    full_DOS = int_value / (2 * pi**2 * t)
    return full_DOS

def kappa_hk_2d(mu, U):
    mu = mpf(mu)
    U = mpf(U)
    if U >= 0:
        return (DOS_HK_2D(mu) + DOS_HK_2D(mu - U)) / 2
    else:
        return DOS_HK_2D(mu - U / 2)

def kappa_landau_2d(rho, mu, U, f_0):
    rho = mpf(rho)
    mu = mpf(mu)
    U = mpf(U)
    f_0 = mpf(f_0)

    kappa_hk_renorm = kappa_hk_2d(mu - f_0 * rho, U)
    denominator = 1 + f_0 * kappa_hk_renorm

    if denominator == 0:
        return mpf('inf')  # or raise an exception
    
    return kappa_hk_renorm / denominator

def create_kappa_list(rho_array, mu_array, U, f_0):
    kappa_list = []

    for rho_val, mu_val in zip(rho_array, mu_array):
        try:
            kappa_i = kappa_landau_2d(rho_val, mu_val, U, f_0)
            kappa_list.append(kappa_i)
        except Exception:
            kappa_list.append(mp.nan)

    return kappa_list

def plot_mpf_arrays(x_array, y_array, xlabel='x', ylabel='y', title='Plot'):
    # Convert mpf elements to regular Python floats for plotting
    x_vals = [float(x) for x in x_array]
    y_vals = [float(y) if y is not None else float('nan') for y in y_array]

    plt.figure(figsize=(8, 5))
    plt.plot(x_vals, y_vals, lw=1.5)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# 2. Test for HK Model